In [1]:
import sqlite3
import pandas as pd
import time
from trading_calendars import get_calendar

def get_stock(code, connection, api):

        try:
            sql = "SELECT * FROM stocks WHERE code = '{}'".format(code)
            df = pd.read_sql(sql, connection, index_col=['code'])
        except:
            df = pd.DataFrame()
    
        if not df.empty:
            return df, True
    
        stock = api.Contracts.Stocks[code]
    
        stock_dict = {
            'code': [stock.code],
            'name': [stock.name],
            'category': [stock.category],
            'day_trade': [stock.day_trade.value]
        }
   
        df = pd.DataFrame(data=stock_dict)
        df = df.set_index('code')
        
        return df, False

ModuleNotFoundError: No module named 'trading_calendars'

In [15]:
def get_ticks(code, date, connection, api):
    
    try:
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND' '{}'".format(code,
                                                                                           date,
                                                                                           date+
                                                                                           datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates=["ts"], index_col=["ts"])
    except:
        df = pd.DataFrame()
        
    if not df.empty:
         return df, True
        
    ticks = api.ticks(api.Contracts.Stocks[code], date.strftime('&Y-&m-&d'))
    
    df = pd.DataFrame({**ticks})
    
    df.ts = pd.to_datatime(df.ts)   
    
    df['code'] = code
    
    df = df.set_index('ts')
    
    return df, False

In [16]:
import shioaji as sj
api = sj.Shioaji()
api.login(
    person_id='N123569782',
    passwd='hons3146',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.


In [17]:
df, in_db = get_ticks('2303',pd.to_datetime('2021/06/15'), connection, api)
    
print(in_db)
df

<SecurityType.Option: 'OPT'> fetch done.
<SecurityType.Stock: 'STK'> fetch done.


Exception: {'status': {'status_code': 422}, 'response': {'detail': [{'loc': ['body', 'ticks_input', 'date'], 'msg': 'Incorrect data format, should be YYYY-MM-DD', 'type': 'value_error'}]}}

In [20]:
def update_ticks(daily_target, connection, api):
    
    main_df = pd.DataFrame()
    
    tw_calendar = get_calendar('XTAI')
    
    for date, codes in daily_target.items():
        
        day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']
        
        print('正在更新{}逐筆成交資料，總共{}檔標的'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes)))   
        
        for code in day_trading_codes:
            
            df, in_db = get_ticks(code, date, connection, api)
            
            if df is not None and not in_db:  
                
                main_df = main_df.append(df, sort=False)
                
                time.sleep(1)
                
            prev_trading_date = tw_calendar.previous_close(date).date()
            
            prev_df, prev_in_db = get_ticks(code, prev_trading_date, connection, api)
                
            if prev_df is not None and not prev_in_db:
                
                    main_df = main_df.append(prev_df, sort=False)
                    
                    time.sleep(1)  
                    
    if not main_df.empty:
        
            main_df.to_sql('ticks', connection, if_exists='append')
            
            return main_df

In [19]:
daily_target = {
        pd.to_datetime('2021-06-09'): ['2303', '2603', '2609', '3576', '6443'],
        pd.to_datetime('2021-06-11'): ['2324', '3576', '6443'],
        pd.to_datetime('2021-06-15'): ['1609', '1714', '1802', '2303', '2344', '2349', '2409', '2603', '3481', '3576', '3704', '6116', '6191']
    }
daily_target

{Timestamp('2021-06-09 00:00:00'): ['2303', '2603', '2609', '3576', '6443'],
 Timestamp('2021-06-11 00:00:00'): ['2324', '3576', '6443'],
 Timestamp('2021-06-15 00:00:00'): ['1609',
  '1714',
  '1802',
  '2303',
  '2344',
  '2349',
  '2409',
  '2603',
  '3481',
  '3576',
  '3704',
  '6116',
  '6191']}

In [18]:
update_ticks(daily_target, connection, api)  

NameError: name 'daily_target' is not defined